# $1^{st}$ part of the submission

import the library I use to open URLs


In [51]:
import urllib.request

#specify which URL/web page I am going to be scraping
#and open the url using urllib.request and put the HTML into the page variable

url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

page = urllib.request.urlopen(url)

#import the BeautifulSoup library so I can parse HTML and XML documents

from bs4 import BeautifulSoup

Then I use Beautiful Soup to parse the HTML data we stored in our ‘url’ variable and store it in a new variable called ‘soup’ in the Beautiful Soup format. 
Jupyter Notebook prefers we specify a parser format so we use the “lxml” library option:
parse the HTML from our URL into

In [52]:
soup = BeautifulSoup(page, "lxml")
soup

#use the 'find_all' function to bring back all instances of the 'table' tag in the HTML and store in 'all_tables' variable

all_tables=soup.find_all("table")
all_tables

right_table=soup.find('table', class_='wikitable sortable')
right_table

#Now I select the table on the website with 3 columns and I assign the column name to all all of them

A=[]
B=[]
C=[]

for row in right_table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==3:
        A.append(cells[0].find(text=True))
        B.append(cells[1].find(text=True))
        C.append(cells[2].find(text=True))

In [53]:
import pandas as pd
df=pd.DataFrame(A,columns=['PostalCode'])
df['Borough']=B
df['Neighborhood']=C
df

#Ignore cells with a borough that is Not assigned

dd=df[df.Borough != 'Not assigned']
dd

#To check that there are no cells with Not assigned in neighborhood

dd_nei = dd.loc[dd['Neighborhood'] == 'Not assigned\n']
dd_nei

#ow I want to remove the '\n' at the end of some string in the Neighbourhood column

dd = dd.replace('\n',' ', regex=True) 
dd

#More than one neighborhood can exist in one postal code area. Rows will be combined into one row with the neighborhoods separated with a comma

dd1=dd.groupby(['PostalCode', 'Borough'])['Neighborhood'].apply(','.join).reset_index()
dd1.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood ,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [41]:
dd1.shape

(103, 3)

# _$2^{nd}$_ part of the submission

I want to add the latitude and longitude of the locations in the table merging the values in dd1 with the values in a cvs file:

In [54]:
# reading csv file 
data = pd.read_csv (r'C:\Users\eleon\Downloads\Geospatial_Coordinates.csv')  
# displying  dataframe - Output 1 
data.head() 
  


,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [55]:
df1 = pd.merge(dd1, data, on='PostalCode')
df1.head ()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood ,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


# $3^{rd}$ part of the submission

Explore and cluster the neighborhoods in Toronto. You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. It is up to you.

Just make sure:

to add enough Markdown cells to explain what you decided to do and to report any observations you make.
to generate maps to visualize your neighborhoods and how they cluster together.

Install folium and geopy.geocoders

In [56]:
conda install -c conda-forge geopy
conda install -c conda-forge folium

SyntaxError: invalid syntax (<ipython-input-56-e35a0753f4ba>, line 1)

In [57]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print('Libraries imported.')


Libraries imported.


To check the number of boroughs and neighborhoods:

In [58]:
print('The dataframe has {} boroughs and {} neighborhood.'.format(
        len(df1['Borough'].unique()),
        df1.shape[0]
    )
)

The dataframe has 10 boroughs and 103 neighborhood.


In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent <em>toronto_explorer</em>, as shown below.

In [59]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


### Create a map of Toronto with neighborhoods superimposed on top.

In [60]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df1['Latitude'], df1['Longitude'], df1['Borough'], df1['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Let's slice the original dataframe and create a new dataframe with only boroughs that contain the word Toronto

In [61]:
etoronto_data = df1[df1['Borough'] == 'East Toronto'].reset_index(drop=True)
etoronto_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West ,Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West ,India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M7Y,East Toronto,Business Reply Mail Processing Centre 969 East...,43.662744,-79.321558


In [62]:
etoronto_data.shape

(5, 5)

Let's get the geographical coordinates of the Borough maned Toronto.

In [63]:
address = 'East Toronto, CA'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of East Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of East Toronto are 43.6247901, -79.3934918.


Let's visualize Toronto and the neighborhoods in it.

In [64]:
# create map of Manhattan using latitude and longitude values
map_etoronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(etoronto_data['Latitude'], etoronto_data['Longitude'], etoronto_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_etoronto)  
    
map_etoronto

I am going to start utilizing the Foursquare API to explore the neighborhoods and segment them.
Define Foursquare Credentials and Version

In [65]:
CLIENT_ID = 'KSJYMEYCFWVXEPGRHRPZDJUIZGDEQY4T0DIJWGJX0YRY4VSJ' # your Foursquare ID
CLIENT_SECRET = '3JVANRI2MPWO3QQV3GMWN1CZEBGK0OZAZFRAFSUQYWUCGBSY' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: KSJYMEYCFWVXEPGRHRPZDJUIZGDEQY4T0DIJWGJX0YRY4VSJ
CLIENT_SECRET:3JVANRI2MPWO3QQV3GMWN1CZEBGK0OZAZFRAFSUQYWUCGBSY


#### Let's explore the first neighborhood in our dataframe.

In [66]:
etoronto_data.loc[0, 'Neighborhood']

'The Beaches'

Get the neighborhood's latitude and longitude values.

In [67]:
neighborhood_latitude = etoronto_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = etoronto_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = etoronto_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of The Beaches are 43.67635739999999, -79.2930312.


let's get the top 100 venues that are in East Toronto within a radius of 500 meters.

In [68]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=KSJYMEYCFWVXEPGRHRPZDJUIZGDEQY4T0DIJWGJX0YRY4VSJ&client_secret=3JVANRI2MPWO3QQV3GMWN1CZEBGK0OZAZFRAFSUQYWUCGBSY&v=20180605&ll=43.67635739999999,-79.2930312&radius=500&limit=100'

In [69]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e64a3f57828ae001bc312ea'},
 'response': {'headerLocation': 'The Beaches',
  'headerFullLocation': 'The Beaches, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 5,
  'suggestedBounds': {'ne': {'lat': 43.680857404499996,
    'lng': -79.28682091449052},
   'sw': {'lat': 43.67185739549999, 'lng': -79.29924148550948}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bd461bc77b29c74a07d9282',
       'name': 'Glen Manor Ravine',
       'location': {'address': 'Glen Manor',
        'crossStreet': 'Queen St.',
        'lat': 43.67682094413784,
        'lng': -79.29394208780985,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.67682094413784,
          'lng': -79.29394208780985}],
        'distanc

In [70]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a pandas dataframe

In [71]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Glen Manor Ravine,Trail,43.676821,-79.293942
1,The Big Carrot Natural Food Market,Health Food Store,43.678879,-79.297734
2,Grover Pub and Grub,Pub,43.679181,-79.297215
3,Upper Beaches,Neighborhood,43.680563,-79.292869
4,Seaspray Restaurant,Asian Restaurant,43.678888,-79.298167


In [72]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

5 venues were returned by Foursquare.


I want to explore the neighborhood in East Toronto

In [73]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Now I run the above function on each neighborhood and create a new dataframe called *etoronto_venues*.

In [74]:
etoronto_venues = getNearbyVenues(names=etoronto_data['Neighborhood'],
                                   latitudes=etoronto_data['Latitude'],
                                   longitudes=etoronto_data['Longitude']
                                  )

The Beaches
The Danforth West ,Riverdale
The Beaches West ,India Bazaar
Studio District 
Business Reply Mail Processing Centre 969 Eastern 


Let's check the size of the resulting dataframe

In [75]:
print(etoronto_venues.shape)
etoronto_venues.head()

(125, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,The Beaches,43.676357,-79.293031,Seaspray Restaurant,43.678888,-79.298167,Asian Restaurant


Let's check how many venues were returned for each neighborhood

In [76]:
etoronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Business Reply Mail Processing Centre 969 Eastern,18,18,18,18,18,18
Studio District,43,43,43,43,43,43
The Beaches,5,5,5,5,5,5
"The Beaches West ,India Bazaar",18,18,18,18,18,18
"The Danforth West ,Riverdale",41,41,41,41,41,41


Let's find out how many unique categories can be curated from all the returned venues

In [77]:
print('There are {} uniques categories.'.format(len(etoronto_venues['Venue Category'].unique())))

There are 66 uniques categories.


## 3. Analyze Each Neighborhood

In [78]:
# one hot encoding
etoronto_onehot = pd.get_dummies(etoronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
etoronto_onehot['Neighborhood'] = etoronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [etoronto_onehot.columns[-1]] + list(etoronto_onehot.columns[:-1])
etoronto_onehot = etoronto_onehot[fixed_columns]

etoronto_onehot.head(50)

,Yoga Studio,American Restaurant,Asian Restaurant,Auto Workshop,Bakery,Bank,Bar,Bookstore,Brewery,Bubble Tea Shop,Burrito Place,Café,Caribbean Restaurant,Cheese Shop,Clothing Store,Coffee Shop,Comfort Food Restaurant,Comic Shop,Convenience Store,Cosmetics Shop,Coworking Space,Dessert Shop,Diner,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gastropub,Gay Bar,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Health Food Store,Ice Cream Shop,Italian Restaurant,Juice Bar,Latin American Restaurant,Light Rail Station,Liquor Store,Middle Eastern Restaurant,Movie Theater,Neighborhood,Park,Pet Store,Pizza Place,Pub,Recording Studio,Restaurant,Sandwich Place,Seafood Restaurant,Skate Park,Smoke Shop,Spa,Stationery Store,Steakhouse,Sushi Restaurant,Thai Restaurant,Thrift / Vintage Store,Trail,Wine Bar
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,The Beaches,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,"The Danforth West ,Riverdale",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"The Danforth West ,Riverdale",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,"The Danforth West ,Riverdale",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,"The Danforth West ,Riverdale",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,"The Danforth West ,Riverdale",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


####  And let's examine the new dataframe size.

In [79]:
etoronto_onehot.shape

(125, 66)

####  Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [80]:
etoronto_grouped = etoronto_onehot.groupby('Neighborhood').mean().reset_index()
etoronto_grouped

,Neighborhood,Yoga Studio,American Restaurant,Asian Restaurant,Auto Workshop,Bakery,Bank,Bar,Bookstore,Brewery,Bubble Tea Shop,Burrito Place,Café,Caribbean Restaurant,Cheese Shop,Clothing Store,Coffee Shop,Comfort Food Restaurant,Comic Shop,Convenience Store,Cosmetics Shop,Coworking Space,Dessert Shop,Diner,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gastropub,Gay Bar,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Health Food Store,Ice Cream Shop,Italian Restaurant,Juice Bar,Latin American Restaurant,Light Rail Station,Liquor Store,Middle Eastern Restaurant,Movie Theater,Park,Pet Store,Pizza Place,Pub,Recording Studio,Restaurant,Sandwich Place,Seafood Restaurant,Skate Park,Smoke Shop,Spa,Stationery Store,Steakhouse,Sushi Restaurant,Thai Restaurant,Thrift / Vintage Store,Trail,Wine Bar
0,Business Reply Mail Processing Centre 969 East...,0.055556,0.000000,0.0,0.055556,0.000000,0.000000,0.000000,0.000000,0.055556,0.00000,0.055556,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.055556,0.000000,0.00000,0.000000,0.00000,0.000000,0.055556,0.055556,0.000000,0.000000,0.00000,0.00000,0.00000,0.055556,0.055556,0.000000,0.000000,0.000000,0.00000,0.000000,0.055556,0.0,0.000000,0.000000,0.00000,0.000000,0.055556,0.000000,0.000000,0.000000,0.055556,0.000000,0.055556,0.000000,0.055556,0.055556,0.000000,0.000000,0.055556,0.055556,0.055556,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
1,Studio District,0.023256,0.046512,0.0,0.000000,0.046512,0.023256,0.023256,0.023256,0.046512,0.00000,0.000000,0.093023,0.00000,0.023256,0.023256,0.069767,0.023256,0.000000,0.023256,0.00000,0.023256,0.00000,0.023256,0.000000,0.000000,0.000000,0.023256,0.00000,0.00000,0.00000,0.000000,0.000000,0.046512,0.023256,0.000000,0.00000,0.000000,0.023256,0.0,0.023256,0.046512,0.00000,0.023256,0.000000,0.000000,0.023256,0.000000,0.023256,0.023256,0.023256,0.000000,0.000000,0.000000,0.023256,0.023256,0.000000,0.000000,0.000000,0.023256,0.000000,0.000000,0.023256,0.023256,0.00000,0.023256
2,The Beaches,0.000000,0.000000,0.2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.2,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.20000,0.000000
3,"The Beaches West ,India Bazaar",0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.055556,0.00000,0.055556,0.000000,0.00000,0.000000,0.000000,0.055556,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.055556,0.055556,0.000000,0.00000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.055556,0.000000,0.0,0.055556,0.055556,0.00000,0.000000,0.000000,0.055556,0.000000,0.055556,0.111111,0.055556,0.055556,0.055556,0.000000,0.000000,0.055556,0.000000,0.000000,0.000000,0.000000,0.000000,0.055556,0.055556,0.000000,0.000000,0.00000,0.000000
4,"The Danforth West ,Riverdale",0.024390,0.024390,0.0,0.000000,0.024390,0.000000,0.000000,0.024390,0.024390,0.02439,0.000000,0.024390,0.02439,0.000000,0.000000,0.097561,0.000000,0.000000,0.000000,0.02439,0.000000,0.02439,0.024390,0.000000,0.000000,0.000000,0.000000,0.02439,0.02439,0.04878,0.000000,0.000000,0.000000,0.000000,0.219512,0.02439,0.000000,0.000000,0.0,0.048780,0.073171,0.02439,0.000000,0.000000,0.024390,0.000000,0.000000,0.000000,0.000000,0.024390,0.024390,0.000000,0.024390,0.000000,0.000000,0.000000,0.000000,0.024390,0.000000,0.000000,0.000000,0.000000,0.000000,0.02439,0.000000


#### Let's confirm the new size

In [81]:
etoronto_grouped.shape

(5, 66)

#### Let's print each neighborhood along with the top 6 most common venues

In [82]:
num_top_venues = 6

for hood in etoronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = etoronto_grouped[etoronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Business Reply Mail Processing Centre 969 Eastern ----
                  venue  freq
0           Yoga Studio  0.06
1            Skate Park  0.06
2         Garden Center  0.06
3                Garden  0.06
4    Light Rail Station  0.06
5  Fast Food Restaurant  0.06


----Studio District ----
                 venue  freq
0                 Café  0.09
1          Coffee Shop  0.07
2              Brewery  0.05
3  American Restaurant  0.05
4   Italian Restaurant  0.05
5            Gastropub  0.05


----The Beaches----
               venue  freq
0                Pub   0.2
1   Asian Restaurant   0.2
2              Trail   0.2
3  Health Food Store   0.2
4          Juice Bar   0.0
5      Movie Theater   0.0


----The Beaches West ,India Bazaar----
          venue  freq
0          Park  0.11
1           Gym  0.06
2  Liquor Store  0.06
3     Pet Store  0.06
4   Pizza Place  0.06
5           Pub  0.06


----The Danforth West ,Riverdale----
                    venue  freq
0        Greek Restauran

#### Let's put that into a *pandas* dataframe

First, let's write a function to sort the venues in descending order

In [83]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood

In [84]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = etoronto_grouped['Neighborhood']

for ind in np.arange(etoronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(etoronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Business Reply Mail Processing Centre 969 East...,Yoga Studio,Skate Park,Gym / Fitness Center,Light Rail Station,Burrito Place,Park,Pizza Place,Brewery,Recording Studio,Restaurant
1,Studio District,Café,Coffee Shop,Gastropub,Brewery,Italian Restaurant,Bakery,American Restaurant,Bar,Bookstore,Bank
2,The Beaches,Health Food Store,Trail,Asian Restaurant,Pub,Farmers Market,Convenience Store,Cosmetics Shop,Coworking Space,Dessert Shop,Diner
3,"The Beaches West ,India Bazaar",Park,Burrito Place,Pub,Gym,Coffee Shop,Ice Cream Shop,Italian Restaurant,Liquor Store,Fish & Chips Shop,Movie Theater
4,"The Danforth West ,Riverdale",Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Furniture / Home Store,Yoga Studio,Pub,Juice Bar,Liquor Store,Trail


#### Cluster Neighborhoods
Run *k*-means to cluster the neighborhood into 5 clusters.

In [85]:
# set number of clusters
kclusters = 5

etoronto_grouped_clustering = etoronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(etoronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 2, 1, 4, 0])

####  Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [91]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

etoronto_merged = etoronto_data

# merge etoronto_grouped with etoronto_data to add latitude/longitude for each neighborhood
etoronto_merged = etoronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

etoronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,etorontoCluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,1,1,Health Food Store,Trail,Asian Restaurant,Pub,Farmers Market,Convenience Store,Cosmetics Shop,Coworking Space,Dessert Shop,Diner
1,M4K,East Toronto,"The Danforth West ,Riverdale",43.679557,-79.352188,0,0,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Furniture / Home Store,Yoga Studio,Pub,Juice Bar,Liquor Store,Trail
2,M4L,East Toronto,"The Beaches West ,India Bazaar",43.668999,-79.315572,4,4,Park,Burrito Place,Pub,Gym,Coffee Shop,Ice Cream Shop,Italian Restaurant,Liquor Store,Fish & Chips Shop,Movie Theater
3,M4M,East Toronto,Studio District,43.659526,-79.340923,2,2,Café,Coffee Shop,Gastropub,Brewery,Italian Restaurant,Bakery,American Restaurant,Bar,Bookstore,Bank
4,M7Y,East Toronto,Business Reply Mail Processing Centre 969 East...,43.662744,-79.321558,3,3,Yoga Studio,Skate Park,Gym / Fitness Center,Light Rail Station,Burrito Place,Park,Pizza Place,Brewery,Recording Studio,Restaurant


Finally, let's visualize the resulting clusters

In [93]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(etoronto_merged['Latitude'], etoronto_merged['Longitude'], etoronto_merged['Neighborhood'], etoronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Now, I can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, I can then assign a name to each cluster.

In [94]:
etoronto_merged.loc[etoronto_merged['Cluster Labels'] == 0, etoronto_merged.columns[[1] + list(range(5, etoronto_merged.shape[1]))]]

,Borough,Cluster Labels,etorontoCluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,East Toronto,0,0,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Furniture / Home Store,Yoga Studio,Pub,Juice Bar,Liquor Store,Trail


In [95]:
etoronto_merged.loc[etoronto_merged['Cluster Labels'] == 1, etoronto_merged.columns[[1] + list(range(5, etoronto_merged.shape[1]))]]

,Borough,Cluster Labels,etorontoCluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,1,1,Health Food Store,Trail,Asian Restaurant,Pub,Farmers Market,Convenience Store,Cosmetics Shop,Coworking Space,Dessert Shop,Diner


In [150]:
etoronto_merged.loc[etoronto_merged['Cluster Labels'] == 2, etoronto_merged.columns[[1] + list(range(5, etoronto_merged.shape[1]))]]

,Borough,etorontoCluster Labels,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,East Toronto,2,2,Café,Coffee Shop,Gastropub,Brewery,Italian Restaurant,Bakery,American Restaurant,Bar,Bookstore,Bank


In [149]:
etoronto_merged.loc[etoronto_merged['Cluster Labels'] == 3, etoronto_merged.columns[[1] + list(range(5, etoronto_merged.shape[1]))]]

,Borough,etorontoCluster Labels,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,East Toronto,3,3,Yoga Studio,Skate Park,Gym / Fitness Center,Light Rail Station,Burrito Place,Park,Pizza Place,Brewery,Recording Studio,Restaurant


In [148]:
etoronto_merged.loc[etoronto_merged['Cluster Labels'] == 4, etoronto_merged.columns[[1] + list(range(5, etoronto_merged.shape[1]))]]

,Borough,etorontoCluster Labels,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,East Toronto,4,4,Park,Burrito Place,Pub,Gym,Coffee Shop,Ice Cream Shop,Italian Restaurant,Liquor Store,Fish & Chips Shop,Movie Theater
